# Demo Overview-Detail imaging on simulated data

Using a ```DemoImspectorConnection``` to simulate microscopy in an existing dataset, you can use this notebook to test autoSTED without a microscope.

In [ ]:
from autosted import AcquisitionPipeline
from autosted.callback_buildingblocks import (JSONSettingsLoader,
                                              LocationKeeper, LocationRemover,
                                              NewestSettingsSelector,
                                              ScanModeSettingsGenerator,
                                              SpiralOffsetGenerator)
from autosted.data import HDF5DataReader
from autosted.detection import SegmentationWrapper
from autosted.imspector.demo_imspector_connection import DemoImspectorConnection
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.utils.coordinate_utils import get_parameter_value_array_from_dict
from autosted.utils.dict_utils import get_parameter_value_array_from_dict
from autosted.utils.parameter_constants import OFFSET_STAGE_GLOBAL_PARAMETERS

# overwrite to old, non-flipped, stage coordinates
# TODO: remove with new test dataset
import autosted.utils.coordinate_utils
autosted.utils.coordinate_utils.DIRECTION_STAGE = (1,1,1)

# setup logging
import logging
logging.basicConfig(level=logging.INFO)


In [ ]:
dataset_path = "/Users/david/Downloads/20241126_stitching_test/e787e527.h5"

dataset = HDF5DataReader(dataset_path)

# get measurement settings & position of first image
first_image_id = next(iter(dataset))
measurement_settings_first = dataset[first_image_id].measurement_settings[0]
start_coords = get_parameter_value_array_from_dict(measurement_settings_first, OFFSET_STAGE_GLOBAL_PARAMETERS)

In [ ]:
from skimage.filters import threshold_otsu
from scipy.ndimage import gaussian_filter, label
from skimage.segmentation import clear_border
from skimage.morphology import dilation, disk, ball


def segment(img):
    # blur and get Otsu threshold
    g = gaussian_filter(img.astype(float), 5)
    t = max(3, threshold_otsu(g))
    # label connected components, remove objects at border, dilate
    labels, _ = label(g > t)
    labels = clear_border(labels)
    labels = dilation(labels, disk(3) if img.ndim==2 else ball(3))
    return labels

In [ ]:
# pipeline with two levels: 'overview', 'detail'
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test",
    hierarchy_levels=["overview", "detail"],
)

# set demo Imspector connection to run on simulated data
# NOTE: needs to be done before starting the run
imspector = DemoImspectorConnection(dataset_path)
pipeline.imspector_connection = imspector

next_overview_generator = AcquisitionTaskGenerator(
    "overview",
    # 1. base settings from file
    JSONSettingsLoader(measurement_settings_first),
    # 2. xy scan mode
    ScanModeSettingsGenerator("xy"),
    # 3. coordinates from spiral
    SpiralOffsetGenerator(
        move_size=[50e-6, 50e-6],
        start_position=start_coords,
    ),
)

# acquisition task generator for details
detail_generator = AcquisitionTaskGenerator(
    "detail",
    # 1. base settings from file
    LocationRemover(JSONSettingsLoader(measurement_settings_first)),
    # 2. locations (stage) from previous (overview) image
    LocationKeeper(NewestSettingsSelector()),
    # 3. set scan mode to xyz
    ScanModeSettingsGenerator("xyz"),
    # 4. segmentation wrapper around segmentation function
    SegmentationWrapper(segment),
)

pipeline.add_callback(next_overview_generator, "overview")
pipeline.add_callback(detail_generator, "overview")

pipeline.add_stopping_condition(
    # maximum number of images per level
    MaximumAcquisitionsStoppingCriterion(
        max_acquisitions_per_level={"overview": 5, "detail": 20}
    )
)

pipeline.run(initial_callback=next_overview_generator)
